In [20]:
import pandas as pd
import requests
import io

# Afficher toutes les colonnes même si nombreuses
pd.set_option('display.max_columns', None)
# pd.reset_option('display.max_columns')

# URL FBref Lens
url = "https://fbref.com/en/squads/fd4e0f7d/Lens-Stats"
headers = {'User-Agent': 'Mozilla/5.0'}

# Télécharger la page et lire tous les tableaux
response = requests.get(url, headers=headers)
response.raise_for_status()
html_io = io.StringIO(response.text)
tables = pd.read_html(html_io)

print(f"✅ {len(tables)} tableaux détectés")


✅ 14 tableaux détectés


In [21]:
df_standard = tables[0].copy()
df_standard.columns = ['_'.join(col).strip() for col in df_standard.columns.values]

# Nettoyage
player_col = 'Unnamed: 0_level_0_Player'
df_standard = df_standard[df_standard[player_col].notna()]
df_standard = df_standard[~df_standard[player_col].str.contains("Squad Total|Opponent Total")]

# Colonnes pertinentes
cols = {
    'Unnamed: 0_level_0_Player':  'Joueur',
    'Unnamed: 1_level_0_Nation':  'Nationalité',
    'Unnamed: 2_level_0_Pos':     'Poste',
    'Unnamed: 3_level_0_Age':     'Âge',
    'Unnamed: 4_level_0_MP':      'Matchs Joués',
    'Playing Time_Min':          'Minutes Jouées',
    'Playing Time_90s':          'Matchs (équiv. 90min)',
    'Performance_Gls':           'Buts',
    'Performance_Ast':           'Passes D',
    'Performance_G+A':           'Buts + Passes',
    'Performance_G-PK':          'Buts (hors pen)',
    'Performance_PK':            'Penaltys Marqués',
    'Performance_PKatt':         'Penaltys Tentés',
    'Performance_CrdY':          'Cartons Jaunes',
    'Performance_CrdR':          'Cartons Rouges',
    'Expected_xG':               'Buts attendus (xG)',               
    'Expected_npxG':             'xG hors penalty (npxG)',           
    'Expected_xAG':              'Passes attendues (xAG)',            
    'Expected_npxG+xAG':         'xG + xAG (buts + passes attendus)',
    'Progression_PrgC':          'Prog. Portées',
    'Progression_PrgP':          'Prog. Passes',
    'Progression_PrgR':          'Prog. Réceptions'
}

# Construction du DataFrame final
df_standard_clean = (
    df_standard[list(cols.keys())]
    .rename(columns=cols)
)

# Aperçu pour validation
from IPython.display import display
display(df_standard_clean.head())

# Export CSV
df_standard_clean.to_csv("../data/standard_stats.csv", index=False)
print("✅ standard_stats.csv créé")

,Joueur,Nationalité,Poste,Âge,Matchs Joués,Minutes Jouées,Matchs (équiv. 90min),Buts,Passes D,Buts + Passes,Buts (hors pen),Penaltys Marqués,Penaltys Tentés,Cartons Jaunes,Cartons Rouges,Buts attendus (xG),xG hors penalty (npxG),Passes attendues (xAG),xG + xAG (buts + passes attendus),Prog. Portées,Prog. Passes,Prog. Réceptions
0,Adrien Thomasson,fr FRA,MF,30.0,33,2615.0,29.1,3.0,7.0,10.0,3.0,0.0,0.0,7.0,0.0,5.9,5.9,6.0,11.9,30.0,149.0,138.0
1,Facundo Medina,ar ARG,DF,25.0,29,2543.0,28.3,0.0,1.0,1.0,0.0,0.0,0.0,15.0,1.0,1.8,1.8,2.0,3.8,46.0,174.0,42.0
2,Andy Diouf,fr FRA,MF,21.0,34,2225.0,24.7,1.0,2.0,3.0,1.0,0.0,0.0,2.0,0.0,2.9,2.9,2.8,5.7,82.0,116.0,94.0
3,Jonathan Gradit,fr FRA,DF,31.0,25,2112.0,23.5,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.8,0.8,0.7,1.5,18.0,127.0,4.0
4,Florian Sotoca,fr FRA,"FW,MF",33.0,31,1794.0,19.9,1.0,1.0,2.0,1.0,0.0,0.0,2.0,1.0,4.6,4.6,1.5,6.1,44.0,105.0,170.0


✅ standard_stats.csv créé


In [22]:
# ── 1. Isoler et fusionner l’index des colonnes ──
df_shoot = tables[4].copy()
df_shoot.columns = ['_'.join(col).strip() for col in df_shoot.columns.values]

# ── 2. Nettoyer les lignes Totaux ──
player_col = 'Unnamed: 0_level_0_Player'
df_shoot = df_shoot[df_shoot[player_col].notna()]
df_shoot = df_shoot[~df_shoot[player_col].str.contains("Squad Total|Opponent Total")]

# ── 3. Sélectionner & renommer les colonnes pertinentes ──
cols = {
    'Unnamed: 0_level_0_Player': 'Joueur',
    'Unnamed: 2_level_0_Pos':    'Poste',
    'Unnamed: 3_level_0_Age':    'Âge',
    'Unnamed: 4_level_0_90s':    'Matchs (éq. 90 min)',
    'Standard_Gls':              'Buts',
    'Standard_Sh':               'Tirs',
    'Standard_SoT':              'Tirs cadrés',
    'Standard_SoT%':             '% tirs cadrés',
    'Standard_Sh/90':            'Tirs / 90 min',
    'Standard_SoT/90':           'Tirs cadrés / 90 min',
    'Standard_G/Sh':             'Buts par tir',
    'Standard_G/SoT':            'Buts par tir cadré',
    'Standard_Dist':             'Distance moyenne du tir',
    'Standard_FK':               'Tirs sur coup francs',
    'Standard_PK':               'Penaltys marqués',
    'Standard_PKatt':            'Penaltys tentés',
    'Expected_xG':               'Buts attendus (xG)',
    'Expected_npxG':             'xG hors pen. (npxG)',
    'Expected_npxG/Sh':          'xG / tir cadré hors pen',
    'Expected_G-xG':             'Buts - xG',
    'Expected_np:G-xG':          'Buts hors pen – xG hors pen'
}

df_shoot_clean = df_shoot[list(cols.keys())].rename(columns=cols)

# ── 4. Aperçu puis export ──
from IPython.display import display
display(df_shoot_clean.head())

df_shoot_clean.to_csv("../data/shooting_stats.csv", index=False)

print("✅ shooting_stats créé")


,Joueur,Poste,Âge,Matchs (éq. 90 min),Buts,Tirs,Tirs cadrés,% tirs cadrés,Tirs / 90 min,Tirs cadrés / 90 min,Buts par tir,Buts par tir cadré,Distance moyenne du tir,Tirs sur coup francs,Penaltys marqués,Penaltys tentés,Buts attendus (xG),xG hors pen. (npxG),xG / tir cadré hors pen,Buts - xG,Buts hors pen – xG hors pen
0,Adrien Thomasson,MF,30.0,29.1,3,41,11,26.8,1.41,0.38,0.07,0.27,14.3,0,0,0,5.9,5.9,0.15,-2.9,-2.9
1,Facundo Medina,DF,25.0,28.3,0,27,3,11.1,0.96,0.11,0.00,0.00,18.3,0,0,0,1.8,1.8,0.07,-1.8,-1.8
2,Andy Diouf,MF,21.0,24.7,1,40,5,12.5,1.62,0.20,0.03,0.20,20.0,0,0,0,2.9,2.9,0.07,-1.9,-1.9
3,Jonathan Gradit,DF,31.0,23.5,0,16,4,25.0,0.68,0.17,0.00,0.00,17.4,0,0,0,0.8,0.8,0.05,-0.8,-0.8
4,Florian Sotoca,"FW,MF",33.0,19.9,1,47,18,38.3,2.36,0.90,0.02,0.06,14.7,0,0,0,4.6,4.6,0.10,-3.6,-3.6


✅ shooting_stats créé


In [23]:
# ── Passes Stats (index 5) ──
df_passes = tables[5].copy()
df_passes.columns = ['_'.join(col).strip() for col in df_passes.columns.values]

# 1) Nettoyer les lignes “Squad Total” / “Opponent Total”
player_col = 'Unnamed: 0_level_0_Player'
df_passes = df_passes[df_passes[player_col].notna()]
df_passes = df_passes[~df_passes[player_col].str.contains("Squad Total|Opponent Total")]

# 2) Sélectionner & renommer les colonnes pertinentes
cols = {
    'Unnamed: 0_level_0_Player': 'Joueur',
    'Unnamed: 1_level_0_Nation': 'Nationalité',
    'Unnamed: 2_level_0_Pos':    'Poste',
    'Unnamed: 3_level_0_Age':    'Âge',
    'Unnamed: 4_level_0_90s':    'Matchs (éq. 90 min)',

    'Total_Cmp':   'Passes réussies',
    'Total_Att':   'Passes tentées',
    'Total_Cmp%':  '% réussite passes',
    'Total_TotDist':  'Distance totale passes',
    'Total_PrgDist':  'Distance progressive passes',

    'Short_Cmp':   'Passes courtes réussies',
    'Short_Att':   'Passes courtes tentées',
    'Medium_Cmp':  'Passes moyennes réussies',
    'Medium_Att':  'Passes moyennes tentées',
    'Long_Cmp':    'Passes longues réussies',
    'Long_Att':    'Passes longues tentées',

    'Unnamed: 19_level_0_Ast':  'Passes décisives',
    'Expected_xA':              'Passes décisives attendues (xA)',
    'Expected_A-xAG':           'Écart passes – xAG',
    'Unnamed: 23_level_0_KP':   'Passes clés',
    'Unnamed: 24_level_0_1/3':  'Passes vers tiers offensif',
    'Unnamed: 25_level_0_PPA':  'Passes vers surface (PPA)',
    'Unnamed: 26_level_0_CrsPA':'Centres vers surface',
    'Unnamed: 27_level_0_PrgP': 'Passes progressives',

    'Unnamed: 28_level_0_Matches': 'Matchs joués'
}

df_passes_clean = df_passes[list(cols.keys())].rename(columns=cols)

# 3) Aperçu et export
from IPython.display import display
print("✅ Passes Stats – aperçu des données nettoyées :")
display(df_passes_clean.head())

df_passes_clean.to_csv("../data/passing_stats.csv", index=False)

✅ Passes Stats – aperçu des données nettoyées :


,Joueur,Nationalité,Poste,Âge,Matchs (éq. 90 min),Passes réussies,Passes tentées,% réussite passes,Distance totale passes,Distance progressive passes,Passes courtes réussies,Passes courtes tentées,Passes moyennes réussies,Passes moyennes tentées,Passes longues réussies,Passes longues tentées,Passes décisives,Passes décisives attendues (xA),Écart passes – xAG,Passes clés,Passes vers tiers offensif,Passes vers surface (PPA),Centres vers surface,Passes progressives,Matchs joués
0,Adrien Thomasson,fr FRA,MF,30.0,29.1,1318,1612,81.8,20594,5103,696,790,472,561,97,142,7,5.2,1.0,56,99,29,7,149,Matches
1,Facundo Medina,ar ARG,DF,25.0,28.3,1655,1889,87.6,27012,9960,827,873,684,744,116,209,1,3.4,-1.0,24,157,30,2,174,Matches
2,Andy Diouf,fr FRA,MF,21.0,24.7,849,1068,79.5,11438,2865,522,597,250,301,34,69,2,3.3,-0.8,33,60,31,3,116,Matches
3,Jonathan Gradit,fr FRA,DF,31.0,23.5,1133,1332,85.1,20473,8262,429,476,610,670,83,160,0,0.9,-0.7,11,109,10,2,127,Matches
4,Florian Sotoca,fr FRA,"FW,MF",33.0,19.9,732,996,73.5,10828,2837,418,495,229,305,50,85,1,2.9,-0.5,18,74,29,9,105,Matches


In [24]:
# ── 1. Isolation & nettoyage des totaux ──
df_creation = tables[7].copy()
df_creation.columns = ['_'.join(col).strip() for col in df_creation.columns.values]

player_col = 'Unnamed: 0_level_0_Player'
df_creation = df_creation[df_creation[player_col].notna()]
df_creation = df_creation[~df_creation[player_col].str.contains("Squad Total|Opponent Total")]

# ── 2. Sélection et renommage des colonnes ──
cols = {
    'Unnamed: 0_level_0_Player': 'Joueur',
    'Unnamed: 2_level_0_Pos':    'Poste',
    'Unnamed: 3_level_0_Age':    'Âge',
    'Unnamed: 4_level_0_90s':    'Matchs (éq. 90 min)',
    
    'SCA_SCA':           'Actions créatrices de tirs (SCA)',
    'SCA_SCA90':         'SCA / 90 min',
    'SCA Types_PassLive':'SCA – passes dans le jeu',
    'SCA Types_PassDead':'SCA – passes sur phases arrêtées',
    'SCA Types_TO':      'SCA – dribbles réussis',
    'SCA Types_Sh':      'SCA – tirs',
    'SCA Types_Fld':     'SCA – fautes provoquées',
    'SCA Types_Def':     'SCA – déviations',
    
    'GCA_GCA':            'Actions créatrices de buts (GCA)',
    'GCA_GCA90':          'GCA / 90 min',
    'GCA Types_PassLive':'GCA – passes vivantes',
    'GCA Types_PassDead':'GCA – passes arrêtées',
    'GCA Types_TO':       'GCA – dribbles réussis',
    'GCA Types_Sh':       'GCA – tirs',
    'GCA Types_Fld':      'GCA – fautes provoquées',
    'GCA Types_Def':      'GCA – déviations',
    
    'Unnamed: 21_level_0_Matches': 'Matchs joués'
}

df_creation_clean = df_creation[list(cols.keys())].rename(columns=cols)

# ── 3. Aperçu & export ──
from IPython.display import display
print("✅ Goal & Shot Creation – aperçu des données nettoyées :")
display(df_creation_clean.head())

df_creation_clean.to_csv("../data/goal_shot_creation.csv", index=False)

✅ Goal & Shot Creation – aperçu des données nettoyées :


,Joueur,Poste,Âge,Matchs (éq. 90 min),Actions créatrices de tirs (SCA),SCA / 90 min,SCA – passes dans le jeu,SCA – passes sur phases arrêtées,SCA – dribbles réussis,SCA – tirs,SCA – fautes provoquées,SCA – déviations,Actions créatrices de buts (GCA),GCA / 90 min,GCA – passes vivantes,GCA – passes arrêtées,GCA – dribbles réussis,GCA – tirs,GCA – fautes provoquées,GCA – déviations,Matchs joués
0,Adrien Thomasson,MF,30.0,29.1,109,3.75,85,9,0,8,3,4,11,0.38,8,0,0,3,0,0,Matches
1,Facundo Medina,DF,25.0,28.3,68,2.41,62,1,1,3,0,1,3,0.11,3,0,0,0,0,0,Matches
2,Andy Diouf,MF,21.0,24.7,85,3.44,63,1,11,7,2,1,3,0.12,3,0,0,0,0,0,Matches
3,Jonathan Gradit,DF,31.0,23.5,25,1.07,23,0,1,1,0,0,0,0.00,0,0,0,0,0,0,Matches
4,Florian Sotoca,"FW,MF",33.0,19.9,53,2.66,44,0,4,2,3,0,3,0.15,3,0,0,0,0,0,Matches


In [25]:
# ── Defensive Actions (index 8) ──
df_def = tables[8].copy()
# 1) Fusionner les multi-index de colonnes
df_def.columns = ['_'.join(col).strip() for col in df_def.columns.values]

# 2) Nettoyer les lignes “Squad Total” / “Opponent Total”
player_col = 'Unnamed: 0_level_0_Player'
df_def = df_def[df_def[player_col].notna()]
df_def = df_def[~df_def[player_col].str.contains("Squad Total|Opponent Total")]

# 3) Sélectionner et renommer les colonnes pertinentes
cols = {
    'Unnamed: 0_level_0_Player':   'Joueur',
    'Unnamed: 1_level_0_Nation':   'Nationalité',
    'Unnamed: 2_level_0_Pos':      'Poste',
    'Unnamed: 3_level_0_Age':      'Âge',
    'Unnamed: 4_level_0_90s':      'Matchs (éq. 90 min)',

    'Tackles_Tkl':       'Tacles réussis',
    'Challenges_Att':    'Duels tentés',
    'Challenges_Tkl%':   '% Duels gagnés',
    'Challenges_Lost':   'Duels perdus',
    'Tackles_Def 3rd':   'Tacles (tiers déf.)',
    'Tackles_Mid 3rd':   'Tacles (tiers méd.)',
    'Tackles_Att 3rd':   'Tacles (tiers off.)',

    'Blocks_Blocks':     'Contres',
    'Blocks_Sh':         'Tirs bloqués',
    'Blocks_Pass':       'Passes bloquées',

    'Unnamed: 17_level_0_Int':     'Interceptions',
    'Unnamed: 18_level_0_Tkl+Int':  'Tacles + interceptions',
    'Unnamed: 19_level_0_Clr':      'Dégagements',
    'Unnamed: 20_level_0_Err':      'Erreurs défensives',

    'Unnamed: 21_level_0_Matches':  'Matchs joués'
}

df_def_clean = df_def[list(cols.keys())].rename(columns=cols)

# 4) Aperçu & export
from IPython.display import display
print("✅ Defensive Actions – aperçu des données nettoyées :")
display(df_def_clean.head())

df_def_clean.to_csv("../data/defensive_actions.csv", index=False)


✅ Defensive Actions – aperçu des données nettoyées :


,Joueur,Nationalité,Poste,Âge,Matchs (éq. 90 min),Tacles réussis,Duels tentés,% Duels gagnés,Duels perdus,Tacles (tiers déf.),Tacles (tiers méd.),Tacles (tiers off.),Contres,Tirs bloqués,Passes bloquées,Interceptions,Tacles + interceptions,Dégagements,Erreurs défensives,Matchs joués
0,Adrien Thomasson,fr FRA,MF,30.0,29.1,90,95,54.7,43,42,25,23,48,3,45,19,109,28,2,Matches
1,Facundo Medina,ar ARG,DF,25.0,28.3,44,42,57.1,18,22,18,4,30,10,20,32,76,70,0,Matches
2,Andy Diouf,fr FRA,MF,21.0,24.7,37,40,40.0,24,10,22,5,24,1,23,15,52,18,2,Matches
3,Jonathan Gradit,fr FRA,DF,31.0,23.5,44,40,57.5,17,22,19,3,27,12,15,25,69,122,2,Matches
4,Florian Sotoca,fr FRA,"FW,MF",33.0,19.9,22,22,54.5,10,3,8,11,12,2,10,14,36,25,0,Matches


In [26]:
# ── Goalkeeper Stats (combine Standard idx2 & Advanced idx3) ──

# 1) Standard
df_gk_std = tables[2].copy()
df_gk_std.columns = ['_'.join(col).strip() for col in df_gk_std.columns.values]
player_col = 'Unnamed: 0_level_0_Player'
df_gk_std = df_gk_std[df_gk_std[player_col].notna()]
df_gk_std = df_gk_std[~df_gk_std[player_col].str.contains("Squad Total|Opponent Total")]

cols_std = {
    'Unnamed: 0_level_0_Player': 'Joueur',
    'Unnamed: 1_level_0_Nation': 'Nationalité',
    'Unnamed: 2_level_0_Pos':    'Poste',
    'Unnamed: 3_level_0_Age':    'Âge',
    'Playing Time_MP':           'Matchs joués',
    'Playing Time_Min':          'Minutes jouées',
    'Playing Time_90s':          'Éq. 90 min',
    'Performance_GA':            'Buts encaissés',
    'Performance_GA90':          'Buts encaissés /90',
    'Performance_SoTA':          'Tirs cadrés subis',
    'Performance_Saves':         'Arrêts',
    'Performance_Save%':         '%Arrêts',
    'Performance_CS':            'Clean sheets',
    'Performance_CS%':           '%Clean sheets'
}
df_gk_std_clean = df_gk_std[list(cols_std)].rename(columns=cols_std)

# 2) Advanced
df_gk_adv = tables[3].copy()
df_gk_adv.columns = ['_'.join(col).strip() for col in df_gk_adv.columns.values]
df_gk_adv = df_gk_adv[df_gk_adv[player_col].notna()]
df_gk_adv = df_gk_adv[~df_gk_adv[player_col].str.contains("Squad Total|Opponent Total")]

cols_adv = {
    'Unnamed: 0_level_0_Player': 'Joueur',       # <— indispensable !
    'Expected_PSxG':      'xG sur tirs cadrés',
    'Expected_PSxG/SoT':  'xG/SoT cadrés',
    'Expected_PSxG+/-':   'Différence buts–xG',
    'Expected_/90':       'xG/90 (cadrés)',
    'Sweeper_#OPA':       'Interventions hors surface',
    'Sweeper_#OPA/90':    'OPA / 90',
    'Sweeper_AvgDist':    'Distance moy. interventions',
    'Crosses_Opp':        'Centres subis',
    'Crosses_Stp':        'Centres arrêtés',
    'Crosses_Stp%':       '% centres arrêtés'
}
df_gk_adv_clean = df_gk_adv[list(cols_adv)].rename(columns=cols_adv)

# 3) Fusion sur 'Joueur'
df_gk = pd.merge(df_gk_std_clean, df_gk_adv_clean, on='Joueur', how='left')

# 4) Aperçu & export
from IPython.display import display
print("✅ Goalkeeper Stats – aperçu des données nettoyées :")
display(df_gk.head())

df_gk.to_csv("../data/goalkeeper_stats.csv", index=False)

✅ Goalkeeper Stats – aperçu des données nettoyées :


,Joueur,Nationalité,Poste,Âge,Matchs joués,Minutes jouées,Éq. 90 min,Buts encaissés,Buts encaissés /90,Tirs cadrés subis,Arrêts,%Arrêts,Clean sheets,%Clean sheets,xG sur tirs cadrés,xG/SoT cadrés,Différence buts–xG,xG/90 (cadrés),Interventions hors surface,OPA / 90,Distance moy. interventions,Centres subis,Centres arrêtés,% centres arrêtés
0,Brice Samba,fr FRA,GK,30.0,15,1350,15.0,14,0.93,54,40,81.5,7,46.7,17.5,0.26,3.5,0.23,22,1.47,16.1,152,17,11.2
1,Mathew Ryan,au AUS,GK,32.0,14,1260,14.0,17,1.21,62,44,75.8,6,42.9,19.5,0.28,2.5,0.18,10,0.71,11.6,176,20,11.4
2,Hervé Koffi,bf BFA,GK,27.0,5,450,5.0,8,1.60,26,18,76.9,0,0.0,11.1,0.38,3.1,0.61,2,0.40,9.4,53,4,7.5


In [27]:
# ── Possession (index 9) ──
df_poss = tables[9].copy()

# 1) Fusionner les multi‐index de colonnes
df_poss.columns = ['_'.join(col).strip() for col in df_poss.columns.values]

# 2) Nettoyer les lignes “Squad Total” / “Opponent Total”
player_col = 'Unnamed: 0_level_0_Player'
df_poss = df_poss[df_poss[player_col].notna()]
df_poss = df_poss[~df_poss[player_col].str.contains("Squad Total|Opponent Total")]

# 3) Sélection et renommage des colonnes pertinentes
cols = {
    'Unnamed: 0_level_0_Player': 'Joueur',
    'Unnamed: 1_level_0_Nation': 'Nationalité',
    'Unnamed: 2_level_0_Pos':    'Poste',
    'Unnamed: 3_level_0_Age':    'Âge',
    'Unnamed: 4_level_0_90s':    'Matchs (équ. 90 min)',

    'Touches_Touches':     'Touches totales',
    'Touches_Def Pen':     'Touches surface défensive',
    'Touches_Def 3rd':     'Touches (tiers défensif)',
    'Touches_Mid 3rd':     'Touches (tiers médian)',
    'Touches_Att 3rd':     'Touches (tiers offensif)',
    'Touches_Att Pen':     'Touches surface adverse',
    'Touches_Live':        'Touches en jeu ouvert',

    'Take-Ons_Att':        'Dribbles tentés',
    'Take-Ons_Succ':       'Dribbles réussis',
    'Take-Ons_Succ%':      '% dribbles réussis',
    'Take-Ons_Tkld':       'Dribbles taclé',
    'Take-Ons_Tkld%':      '% dribbles taclé',

    'Carries_Carries':     'Portées totales',
    'Carries_TotDist':     'Distance portée totale',
    'Carries_PrgDist':     'Distance portées progressives',
    'Carries_PrgC':        'Portées progressives',
    'Carries_1/3':         'Portées vers tiers offensif',
    'Carries_CPA':         'Portées dans surface adverse',
    'Carries_Mis':         'Mauvais contrôles',
    'Carries_Dis':         'Ballons perdus',

    'Receiving_Rec':       'Réceptions totales',
    'Receiving_PrgR':      'Réceptions progressives',

    'Unnamed: 27_level_0_Matches': 'Matchs joués'
}

df_poss_clean = df_poss[list(cols.keys())].rename(columns=cols)

# 4) Aperçu et export CSV
from IPython.display import display
print("✅ Possession – aperçu des données nettoyées :")
display(df_poss_clean.head())

df_poss_clean.to_csv("../data/possession_stats.csv", index=False)


✅ Possession – aperçu des données nettoyées :


,Joueur,Nationalité,Poste,Âge,Matchs (équ. 90 min),Touches totales,Touches surface défensive,Touches (tiers défensif),Touches (tiers médian),Touches (tiers offensif),Touches surface adverse,Touches en jeu ouvert,Dribbles tentés,Dribbles réussis,% dribbles réussis,Dribbles taclé,% dribbles taclé,Portées totales,Distance portée totale,Distance portées progressives,Portées progressives,Portées vers tiers offensif,Portées dans surface adverse,Mauvais contrôles,Ballons perdus,Réceptions totales,Réceptions progressives,Matchs joués
0,Adrien Thomasson,fr FRA,MF,30.0,29.1,1995,52,333,1028,650,112,1995,60,26,43.3,32,53.3,1056,4501,2078,30,34,9,67,41,1361,138,Matches
1,Facundo Medina,ar ARG,DF,25.0,28.3,2176,121,706,1150,341,31,2176,27,10,37.0,17,63.0,1328,6893,3678,46,48,2,27,14,1514,42,Matches
2,Andy Diouf,fr FRA,MF,21.0,24.7,1367,23,158,741,505,51,1367,87,43,49.4,37,42.5,751,5197,2776,82,74,10,75,42,966,94,Matches
3,Jonathan Gradit,fr FRA,DF,31.0,23.5,1594,159,629,854,117,22,1594,9,3,33.3,5,55.6,865,3570,1960,18,15,0,11,3,1086,4,Matches
4,Florian Sotoca,fr FRA,"FW,MF",33.0,19.9,1186,25,105,546,544,89,1186,39,16,41.0,19,48.7,679,2942,1488,44,43,7,20,22,932,170,Matches


In [28]:
# ── Playing Time (index 10) ──
df_play = tables[10].copy()

# 1) Fusionner les multi‐index de colonnes
df_play.columns = ['_'.join(col).strip() for col in df_play.columns.values]

# 2) Nettoyer les lignes “Squad Total” / “Opponent Total”
player_col = 'Unnamed: 0_level_0_Player'
df_play = df_play[df_play[player_col].notna()]
df_play = df_play[~df_play[player_col].str.contains("Squad Total|Opponent Total")]

# 3) Sélection et renommage des colonnes pertinentes
cols = {
    'Unnamed: 0_level_0_Player':   'Joueur',
    'Unnamed: 1_level_0_Nation':   'Nationalité',
    'Unnamed: 2_level_0_Pos':      'Poste',
    'Unnamed: 3_level_0_Age':      'Âge',
    'Unnamed: 4_level_0_MP':       'Matchs joués',
    'Playing Time_Min':            'Minutes jouées',
    'Playing Time_Mn/MP':          'Min / match',
    'Playing Time_Min%':           '% Minutes jouées',
    'Playing Time_90s':            'Éq. 90 min',
    'Starts_Starts':               'Titulaire',
    'Starts_Mn/Start':             'Min / titularisation',
    'Starts_Compl':                'Matchs complétés',
    'Subs_Subs':                   'Entrées en jeu',
    'Subs_Mn/Sub':                 'Min / entrée',
    'Subs_unSub':                  'Pas remplacé après entrée',
    'Team Success_PPM':            'Points / match',
    'Team Success_onG':            'Buts marqués (ON)',
    'Team Success_onGA':           'Buts encaissés (ON)',
    'Team Success_+/-':            'Différence de buts (ON)',
    'Team Success_+/-90':          'Différence / 90 min (ON)',
    'Team Success_On-Off':         'Impact On – Off',
    'Team Success (xG)_onxG':      'xG marqués (ON)',
    'Team Success (xG)_onxGA':     'xG encaissés (ON)',
    'Team Success (xG)_xG+/-':     'Différence xG (ON)',
    'Team Success (xG)_xG+/-90':   'Différence xG / 90 min (ON)',
    'Team Success (xG)_On-Off':    'Impact xG On – Off',
    'Unnamed: 26_level_0_Matches': 'Matchs joués'
}

df_play_clean = df_play[list(cols.keys())].rename(columns=cols)

# 4) Aperçu & export
from IPython.display import display
print("✅ Playing Time – aperçu des données nettoyées :")
display(df_play_clean.head())

df_play_clean.to_csv("../data/playing_time_stats.csv", index=False)

✅ Playing Time – aperçu des données nettoyées :


,Joueur,Nationalité,Poste,Âge,Matchs joués,Minutes jouées,Min / match,% Minutes jouées,Éq. 90 min,Titulaire,Min / titularisation,Matchs complétés,Entrées en jeu,Min / entrée,Pas remplacé après entrée,Points / match,Buts marqués (ON),Buts encaissés (ON),Différence de buts (ON),Différence / 90 min (ON),Impact On – Off,xG marqués (ON),xG encaissés (ON),Différence xG (ON),Différence xG / 90 min (ON),Impact xG On – Off,Matchs joués
0,Adrien Thomasson,fr FRA,MF,30.0,33,2615.0,79.0,85.5,29.1,31,83.0,18,2,27.0,0,1.48,36.0,32.0,4.0,0.14,0.34,45.2,37.7,7.4,0.26,0.44,Matches
1,Facundo Medina,ar ARG,DF,25.0,29,2543.0,88.0,83.1,28.3,29,88.0,24,0,NaN,0,1.69,39.0,30.0,9.0,0.32,1.36,45.6,33.5,12.1,0.43,1.39,Matches
2,Andy Diouf,fr FRA,MF,21.0,34,2225.0,65.0,72.7,24.7,27,77.0,8,7,22.0,0,1.53,27.0,23.0,4.0,0.16,0.27,36.4,31.9,4.5,0.18,-0.03,Matches
3,Jonathan Gradit,fr FRA,DF,31.0,25,2112.0,84.0,69.0,23.5,24,87.0,19,1,15.0,4,1.52,27.0,29.0,-2.0,-0.09,-0.56,34.1,32.4,1.7,0.07,-0.38,Matches
4,Florian Sotoca,fr FRA,"FW,MF",33.0,31,1794.0,58.0,58.6,19.9,22,74.0,7,9,20.0,0,1.45,18.0,21.0,-3.0,-0.15,-0.58,30.4,26.2,4.2,0.21,0.04,Matches


In [29]:
# 1) Charger le fichier standard
df_std = pd.read_csv("../data/standard_stats.csv")

# 2) Sélectionner les colonnes de base
df_players = df_std[['Joueur', 'Nationalité', 'Poste', 'Âge']].copy()

# 3) Exporter le CSV
df_players.to_csv("../data/players.csv", index=False)

# Aperçu
df_players.head()


,Joueur,Nationalité,Poste,Âge
0,Adrien Thomasson,fr FRA,MF,30.0
1,Facundo Medina,ar ARG,DF,25.0
2,Andy Diouf,fr FRA,MF,21.0
3,Jonathan Gradit,fr FRA,DF,31.0
4,Florian Sotoca,fr FRA,"FW,MF",33.0


In [34]:
from pathlib import Path

# Colonnes à supprimer sur tous les fichiers (sauf players.csv)
cols_to_drop = [
    "Poste",
    "Âge",
    "Matchs Joués",
    "Matchs joués",
    "Minutes Jouées",
    "Matchs (équiv. 90min)",
    "Matchs joués.1"
]

# Dossier où sont rangés tes CSV
data_dir = Path(".")

# Parcours de tous les .csv sauf players.csv
for csv_path in data_dir.glob("../data/*.csv"):
    if csv_path.name == "../data/players.csv":
        continue

    # Chargement
    df = pd.read_csv(csv_path)

    # On supprime les colonnes présentes dans cols_to_drop
    to_drop = [c for c in cols_to_drop if c in df.columns]
    if to_drop:
        df = df.drop(columns=to_drop)
        print(f"→ {csv_path.name} : colonnes supprimées -> {to_drop}")
    else:
        print(f"→ {csv_path.name} : aucune colonne à supprimer")

    # Écrasement du CSV
    df.to_csv(csv_path, index=False)


→ standard_stats.csv : aucune colonne à supprimer
→ playing_time_stats.csv : colonnes supprimées -> ['Matchs joués']
→ defensive_actions.csv : colonnes supprimées -> ['Matchs joués']
→ shooting_stats.csv : aucune colonne à supprimer
→ possession_stats.csv : colonnes supprimées -> ['Matchs joués']
→ goal_shot_creation.csv : colonnes supprimées -> ['Matchs joués']
→ goalkeeper_stats.csv : colonnes supprimées -> ['Matchs joués']
→ miscellaneous_stats.csv : colonnes supprimées -> ['Matchs joués']
→ pass_types_stats.csv : colonnes supprimées -> ['Matchs joués']
→ passing_stats.csv : colonnes supprimées -> ['Matchs joués']
→ players.csv : aucune colonne à supprimer


In [35]:
test = pd.read_csv('../data/defensive_actions.csv')
test

,Joueur,Nationalité,Matchs (éq. 90 min),Tacles réussis,Duels tentés,% Duels gagnés,Duels perdus,Tacles (tiers déf.),Tacles (tiers méd.),Tacles (tiers off.),Contres,Tirs bloqués,Passes bloquées,Interceptions,Tacles + interceptions,Dégagements,Erreurs défensives
0,Adrien Thomasson,fr FRA,29.1,90,95,54.7,43,42,25,23,48,3,45,19,109,28,2
1,Facundo Medina,ar ARG,28.3,44,42,57.1,18,22,18,4,30,10,20,32,76,70,0
2,Andy Diouf,fr FRA,24.7,37,40,40.0,24,10,22,5,24,1,23,15,52,18,2
3,Jonathan Gradit,fr FRA,23.5,44,40,57.5,17,22,19,3,27,12,15,25,69,122,2
4,Florian Sotoca,fr FRA,19.9,22,22,54.5,10,3,8,11,12,2,10,14,36,25,0
5,Deiver Machado,co COL,20.1,36,30,56.7,13,14,14,8,20,4,16,16,52,34,2
6,Neil El Aynaoui,fr FRA,17.6,39,34,35.3,22,18,15,6,34,8,26,30,69,29,0
7,M'Bala Nzola,ao ANG,16.2,5,9,33.3,6,0,1,4,9,1,8,5,10,7,0
8,Anass Zaroury,ma MAR,15.6,23,26,34.6,17,6,12,5,9,0,9,6,29,15,0
9,Przemysław Frankowski,pl POL,15.4,32,30,63.3,11,17,10,5,10,2,8,19,51,28,0
